<br>


# CATI



- [CATI disponibiliza mapeamento de áreas urbanas para gestores públicos](https://www.cati.sp.gov.br/portal/imprensa/noticia/cati-disponibiliza-mapeamento-de-areas-urbanas-para-gestores-publicos)


Mapa com Delimitação de áreas urbanizadas do Estado de São Paulo.

> O trabalho desenvolvido pela equipe do Ciagro foi realizado sobre as ortofotos do projeto “Mapeia São Paulo”, disponibilizado pela Empresa Paulista de Planejamento Metropolitano-S/A (Emplasa), tendo como base os limites municipais oficiais do Instituto Geográfico e Cartográfico do Estado de São Paulo (IGC). “Foram mapeados 842.750,1 hectares, que somados aos 20.331.149,2 hectares, quantificados pelo Levantamento Censitário de Unidades de Produção Agropecuária (Lupa/CATI), representam 85,2% do território paulista”, informa Mário Ivo, acrescentando: “esperamos que esse material subsidie diferentes trabalhos aos munícipes do Estado, embase Planos Diretores e Leis Orgânicas, auxiliando estudos sobre as consequências da pressão urbana sobre o meio rural e seus recursos ambientais. Esperamos também que ele propicie a geração de políticas de incentivo à manutenção do produtor no meio rural, como um dos principais atores da conservação ambiental”.

In [ ]:
import io
import tempfile
from pathlib import Path
from zipfile import ZipFile

import geopandas as gpd
import rarfile
import requests

import open_geodata as geo

In [ ]:
db = geo.data.DB(db='sp_cati')
db.list_data

In [ ]:
# Problemas com SSL
# gdf = geo.load_dataset(db='sp_cati', name='geo.urbanizado_2016')
# gdf

In [ ]:
url = db.json['geo.urbanizado_2016']['url']
url

In [ ]:
# COm uma pasta temporária
with tempfile.TemporaryDirectory() as temp_path:
    # Path
    temp_path = Path(temp_path)

    # Faz o download
    with requests.get(url, verify=False) as r:

        # Dentro do zip
        with ZipFile(io.BytesIO(r.content)) as zip_ref:
            for info in zip_ref.infolist():
                print(info.filename)

                # Lê o .rar
                if info.filename == 'Resultado_Final.rar':
                    rar_bytes = io.BytesIO(zip_ref.read(info.filename))
                    with rarfile.RarFile(file=rar_bytes) as rar_obj:
                        rar_obj.extractall(temp_path)

                    # Lê o geodataframe
                    gdf = gpd.read_file(
                        filename=list(temp_path.rglob('*.shp'))[0]
                    )

# Resultado
gdf.head()

In [ ]:
m = gdf.explore(column='EDR', legend=False)
m